In [6]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
from ultralytics import YOLO
import xml.etree.ElementTree as ET
import os

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Shamim\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
%pip install opencv-python

     -------------------------------------- 39.5/39.5 MB 862.0 kB/s eta 0:00:00
     -------------------------------------- 12.9/12.9 MB 742.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install ultralytics

^C


  Using cached ultralytics-8.3.92-py3-none-any.whl (949 kB)
  Using cached numpy-2.1.1-cp311-cp311-win_amd64.whl (12.9 MB)
  Using cached matplotlib-3.10.1-cp311-cp311-win_amd64.whl (8.1 MB)
  Using cached pillow-11.1.0-cp311-cp311-win_amd64.whl (2.6 MB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl (161 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached scipy-1.15.2-cp311-cp311-win_amd64.whl (41.2 MB)
     ------------------------------------ 204.2/204.2 MB 553.0 kB/s eta 0:00:00
  Using cached torchvision-0.21.0-cp311-cp311-win_amd64.whl (1.6 MB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
  Using cached contourpy-1.3.1-cp311-cp311-win_amd64.whl (219 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Shamim\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_internal\cli\base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "C:\Users\Shamim\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_internal\cli\req_command.py", line 247, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Shamim\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_internal\commands\install.py", line 400, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "C:\Users\Shamim\AppData\Local\Programs\Python\Python311\Lib\site-packages\pip\_internal\resolution\resolvelib\resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "C:\Users\Shamim\AppData\Local\Programs\Python\Python311\Lib

In [7]:
train_path=str(os.getcwd())+"/dataset/images/train/"
val_path=str(os.getcwd())+"/dataset/images/val/"
test_path=str(os.getcwd())+"/dataset/images/test/"
anno_train_path=str(os.getcwd())+"/dataset/labels/train/"
anno_val_path=str(os.getcwd())+"/dataset/labels/val/"
anno_test_path=str(os.getcwd())+"/dataset/labels/test/"
image_path=str(os.getcwd())+"/JPEGImages/"
anno_path=str(os.getcwd())+"/Annotations/"

In [8]:
def applyCLAHEandSobel(image_path,savepath):
    img = cv2.imread(image_path)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)
    cl2 = clahe.apply(a)
    cl3 = clahe.apply(b)
    limg = cv2.merge((cl, cl2, cl3))
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    gray = cv2.cvtColor(enhanced_img, cv2.COLOR_RGB2GRAY)
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)  # Detect vertical edges
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)  # Detect horizontal edges
    sobel_edges = np.sqrt(sobel_x**2 + sobel_y**2)
    sobel_edges = cv2.normalize(sobel_edges, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    sobel_edges_colored = cv2.cvtColor(sobel_edges, cv2.COLOR_GRAY2BGR)
    sharpened = cv2.addWeighted(enhanced_img, 1, sobel_edges_colored, 0.5, 0)
    sharpened = cv2.cvtColor(sharpened, cv2.COLOR_BGR2RGB)
    sharpened = cv2.fastNlMeansDenoisingColored(sharpened, None, 10, 10, 7, 21)
    cv2.imwrite(savepath, sharpened)

In [9]:
def convert_voc_to_yolo(xml_path, output_txt_path, class_dict, orig_width=640, orig_height=480, new_size=640):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    scale = new_size / max(orig_width, orig_height)  # Scaling factor
    pad_x = (new_size - orig_width * scale) / 2  # Horizontal padding
    pad_y = (new_size - orig_height * scale) / 2  # Vertical padding
    with open(output_txt_path, "w") as f:
        for obj in root.findall("object"):
            class_name = obj.find("name").text
            if class_name not in class_dict:
                continue
            class_id = class_dict[class_name]
            bbox = obj.find("bndbox")
            x_min = int(bbox.find("xmin").text)
            y_min = int(bbox.find("ymin").text)
            x_max = int(bbox.find("xmax").text)
            y_max = int(bbox.find("ymax").text)
            x_min = (x_min * scale + pad_x) / new_size
            y_min = (y_min * scale + pad_y) / new_size
            x_max = (x_max * scale + pad_x) / new_size
            y_max = (y_max * scale + pad_y) / new_size
            x_center = (x_min + x_max) / 2
            y_center = (y_min + y_max) / 2
            bbox_width = x_max - x_min
            bbox_height = y_max - y_min
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

In [10]:
with open("train.txt", "r") as file:
    content = file.read()
train_list=content.split("\n")
with open("val.txt", "r") as file:
    content = file.read()
val_list=content.split("\n")
with open("test.txt", "r") as file:
    content = file.read()
test_list=content.split("\n")

In [11]:
class_dict = {"WBC": 0, "RBC": 1, "Platelets":2}
for i in train_list:
    path_img=image_path+i+".jpg"
    path_anno=anno_path+i+".xml"
    save_img_path=train_path+i+".jpg"
    save_anno_path=anno_train_path+i+".txt"
    applyCLAHEandSobel(path_img,save_img_path)
    convert_voc_to_yolo(path_anno,save_anno_path,class_dict)
for i in val_list:
    path_img=image_path+i+".jpg"
    path_anno=anno_path+i+".xml"
    save_img_path=val_path+i+".jpg"
    save_anno_path=anno_val_path+i+".txt"
    applyCLAHEandSobel(path_img,save_img_path)
    convert_voc_to_yolo(path_anno,save_anno_path,class_dict)
for i in test_list:
    path_img=image_path+i+".jpg"
    path_anno=anno_path+i+".xml"
    save_img_path=test_path+i+".jpg"
    save_anno_path=anno_test_path+i+".txt"
    applyCLAHEandSobel(path_img,save_img_path)
    convert_voc_to_yolo(path_anno,save_anno_path,class_dict)